# Kubeflow Fairing Introduction

Kubeflow Fairing is a Python package that streamlines the process of `building`, `training`, and `deploying` machine learning (ML) models in a hybrid cloud environment. By using Kubeflow Fairing and adding a few lines of code, you can run your ML training job locally or in the cloud, directly from Python code or a Jupyter notebook. After your training job is complete, you can use Kubeflow Fairing to deploy your trained model as a prediction endpoint.


# How does Kubeflow Fairing work

Kubeflow Fairing 
1. Packages your Jupyter notebook, Python function, or Python file as a Docker image
2. Deploys and runs the training job on Kubeflow or AI Platform. 
3. Deploy your trained model as a prediction endpoint on Kubeflow after your training job is complete.


# Goals of Kubeflow Fairing project

- Easily package ML training jobs: Enable ML practitioners to easily package their ML model training code, and their code’s dependencies, as a Docker image.
- Easily train ML models in a hybrid cloud environment: Provide a high-level API for training ML models to make it easy to run training jobs in the cloud, without needing to understand the underlying infrastructure.
- Streamline the process of deploying a trained model: Make it easy for ML practitioners to deploy trained ML models to a hybrid cloud environment.

## Train and deploy model on Kubeflow in Notebooks

This examples comes from a upstream fairing [example](https://github.com/kubeflow/fairing/tree/master/examples/prediction).


Please check Kaggle competiton [
House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)
for details about the ML problem we want to resolve.

This notebook introduces you to using Kubeflow Fairing to train and deploy a model to Kubeflow on Amazon EKS. This notebook demonstrate how to:

* Train an XGBoost model in a local notebook,
* Use Kubeflow Fairing to train an XGBoost model remotely on Kubeflow,
* Use Kubeflow Fairing to deploy a trained model to Kubeflow,
* Call the deployed endpoint for predictions.


### Install python dependencies

In [1]:
# Install latest Fairing from github repository
!git clone https://github.com/kubeflow/fairing
!pip install --upgrade fairing/

fatal: destination path 'fairing' already exists and is not an empty directory.
Processing ./fairing


  Stored in directory: /tmp/pip-ephem-wheel-cache-gmng49xp/wheels/98/1e/af/74029ca3fc6818382f1a3aa55f499bdab9e832420bf6a5f7c4
Successfully built kubeflow-fairing
  Found existing installation: kubeflow-fairing 0.7.1
    Uninstalling kubeflow-fairing-0.7.1:
      Successfully uninstalled kubeflow-fairing-0.7.1
You are using pip version 19.0.1, however version 20.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%%writefile requirements.txt
pandas
joblib
numpy
xgboost
scikit-learn>=0.21.0
seldon-core
tornado>=6.0.3

Overwriting requirements.txt


In [3]:
!pip install -r requirements.txt

You are using pip version 19.0.1, however version 20.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Train on Kubernetes

We will show you how to run a training job in kubernetes cluster. You can use `ECR` as your container image registry.

In [4]:
# Authenticate ECR
# This command retrieves a token that is valid for a specified registry for 12 hours, 
# and then it prints a docker login command with that authorization token. 
# Then we executate this command to login ECR

REGION='us-west-2'
!eval $(aws ecr get-login --no-include-email --region=$REGION)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
# Create an ECR repository in the same region
# If you receive "RepositoryAlreadyExistsException" error, it means the repository already
# exists. You can move to the next step
!aws ecr create-repository --repository-name fairing-job --region=$REGION


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'fairing-job' already exists in the registry with id '123384328431'


If you see any issues below, please restart notebook. It's probably because of new installed packages.

Click `Kernel` -> `Restart & Clear Output`

### Develop your model

In [6]:
import argparse
import logging
import joblib
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

logging.basicConfig(format='%(message)s')
logging.getLogger().setLevel(logging.INFO)

In [7]:
def read_input(file_name, test_size=0.25):
    """Read input data and split it into train and test."""
    data = pd.read_csv(file_name)
    data.dropna(axis=0, subset=['SalePrice'], inplace=True)

    y = data.SalePrice
    X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

    train_X, test_X, train_y, test_y = train_test_split(X.values,
                                                      y.values,
                                                      test_size=test_size,
                                                      shuffle=False)

    imputer = SimpleImputer()
    train_X = imputer.fit_transform(train_X)
    test_X = imputer.transform(test_X)

    return (train_X, train_y), (test_X, test_y)

def train_model(train_X,
                train_y,
                test_X,
                test_y,
                n_estimators,
                learning_rate):
    """Train the model using XGBRegressor."""
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)

    model.fit(train_X,
            train_y,
            early_stopping_rounds=40,
            eval_set=[(test_X, test_y)])

    print("Best RMSE on eval: %.2f with %d rounds" %
               (model.best_score,
                model.best_iteration+1))
    return model

def eval_model(model, test_X, test_y):
    """Evaluate the model performance."""
    predictions = model.predict(test_X)
    logging.info("mean_absolute_error=%.2f", mean_absolute_error(predictions, test_y))

def save_model(model, model_file):
    """Save XGBoost model for serving."""
    joblib.dump(model, model_file)
    logging.info("Model export success: %s", model_file)
    
    
class HousingServe(object):
    
    def __init__(self):
        self.train_input = "ames_dataset/train.csv"
        self.n_estimators = 50
        self.learning_rate = 0.1
        self.model_file = "trained_ames_model.dat"
        self.model = None

    def train(self):
        (train_X, train_y), (test_X, test_y) = read_input(self.train_input)
        model = train_model(train_X,
                          train_y,
                          test_X,
                          test_y,
                          self.n_estimators,
                          self.learning_rate)

        eval_model(model, test_X, test_y)
        save_model(model, self.model_file)

    def predict(self, X, feature_names=None):
        """Predict using the model for given ndarray."""
        if not self.model:
            self.model = joblib.load(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)
        # Do any postprocessing
        return prediction

### Train an XGBoost model in a notebook

In [8]:
model = HousingServe()
model.train()

[21:14:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:177514
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:161858
[2]	validation_0-rmse:147237
[3]	validation_0-rmse:134132
[4]	validation_0-rmse:122224
[5]	validation_0-rmse:111538
[6]	validation_0-rmse:102142
[7]	validation_0-rmse:93392.3
[8]	validation_0-rmse:85824.6
[9]	validation_0-rmse:79667.6
[10]	validation_0-rmse:73463.4
[11]	validation_0-rmse:68059.4
[12]	validation_0-rmse:63350.5
[13]	validation_0-rmse:59732.1
[14]	validation_0-rmse:56260.7
[15]	validation_0-rmse:53392.6
[16]	validation_0-rmse:50770.8
[17]	validation_0-rmse:48107.8
[18]	validation_0-rmse:45923.9
[19]	validation_0-rmse:44154.2
[20]	validation_0-rmse:42488.1
[21]	validation_0-rmse:41263.3
[22]	validation_0-rmse:40212.8
[23]	validation_0-rmse:39089.1
[24]	validation_0-rmse:37691.1
[25]	validation_0-rmse:36875.2
[26]	validation_0-r

mean_absolute_error=18173.15
Model export success: trained_ames_model.dat


Best RMSE on eval: 28787.72 with 50 rounds


### Set up Kubeflow Fairing for training and predictions

_Note: remember to change `<your-globally-unique-bucket-name>` to your own s3 bucket._

In [9]:
from kubeflow import fairing
from kubeflow.fairing import TrainJob
from kubeflow.fairing.backends import KubeflowAWSBackend


from kubeflow import fairing

FAIRING_BACKEND = 'KubeflowAWSBackend'

AWS_ACCOUNT_ID = fairing.cloud.aws.guess_account_id()
AWS_REGION = 'us-west-2'
DOCKER_REGISTRY = '{}.dkr.ecr.{}.amazonaws.com'.format(AWS_ACCOUNT_ID, AWS_REGION)

##############
S3_BUCKET = 'chuba-ml-bucket'
##############

In [10]:
import importlib

if FAIRING_BACKEND == 'KubeflowAWSBackend':
    from kubeflow.fairing.builders.cluster.s3_context import S3ContextSource
    BuildContext = S3ContextSource(
        aws_account=AWS_ACCOUNT_ID, region=AWS_REGION,
        bucket_name=S3_BUCKET
    )

BackendClass = getattr(importlib.import_module('kubeflow.fairing.backends'), FAIRING_BACKEND)

### Train an XGBoost model remotely on Kubeflow
Import the `TrainJob` and use the configured backend class. Kubeflow Fairing packages the `HousingServe` class, the training data, and the training job's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the training job on Kubeflow.


In [11]:
from kubeflow.fairing import TrainJob
train_job = TrainJob(HousingServe, input_files=['ames_dataset/train.csv', "requirements.txt"],
                     docker_registry=DOCKER_REGISTRY,
                     backend=BackendClass(build_context_source=BuildContext))
train_job.submit()

Using default base docker image: registry.hub.docker.com/library/python:3.6.8
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/opt/conda/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_ezmqec5k
/opt/conda/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-qbqx6-62xnn to start...
Waiting for fairing-builder-qbqx6-62xnn to start...
Waiting for fairing-builder-qbqx6-62xnn to start...
Pod started running True


INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0001] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0001] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0002] Built cross stage deps: map[]
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0003] Unpacking rootfs as cm

  Stored in directory: /tmp/pip-ephem-wheel-cache-vvi4von_/wheels/84/bf/40/2f6ef700f48401ca40e5e3dd7d0e3c0a90e064897b7fe5fc08
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvi4von_/wheels/7b/dc/25/3cf0b35c129232ee596c413f13d1d1f5a8e38c427266276dfd
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvi4von_/wheels/f2/84/7f/e89da3ee8ce35598d6382b6389fa2ada5d66acca2422537994
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvi4von_/wheels/93/e9/b5/1cdc3544f99a54caca13832b5afa26fd98701fe709dc049576
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvi4von_/wheels/e4/76/4d/a95b8dd7b452b69e8ed4f68b69e1b55e12c9c9624dd962b191
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvi4von_/wheels/d7/7a/30/d212623a4cd34f6cce400f8122b1b7af740d3440c68023d51f
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvi4von_/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvi4von_/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a4

You should consider upgrading via the 'pip install --upgrade pip' command.
INFO[0054] Taking snapshot of full filesystem...
INFO[0071] Using files from context: [/kaniko/buildcontext/app]
INFO[0071] COPY /app/ /app/
INFO[0071] Taking snapshot of files...


The job fairing-job-rk6wl launched.
Waiting for fairing-job-rk6wl-r5skx to start...
Waiting for fairing-job-rk6wl-r5skx to start...
Waiting for fairing-job-rk6wl-r5skx to start...
Pod started running True


[21:16:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:177514
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:161858
[2]	validation_0-rmse:147237
[3]	validation_0-rmse:134132
[4]	validation_0-rmse:122224
[5]	validation_0-rmse:111538
[6]	validation_0-rmse:102142
[7]	validation_0-rmse:93392.3
[8]	validation_0-rmse:85824.6
[9]	validation_0-rmse:79667.6
[10]	validation_0-rmse:73463.4
[11]	validation_0-rmse:68059.4
[12]	validation_0-rmse:63350.5
[13]	validation_0-rmse:59732.1
[14]	validation_0-rmse:56260.7
[15]	validation_0-rmse:53392.6
[16]	validation_0-rmse:50770.8
[17]	validation_0-rmse:48107.8
[18]	validation_0-rmse:45923.9
[19]	validation_0-rmse:44154.2
[20]	validation_0-rmse:42488.1
[21]	validation_0-rmse:41263.3
[22]	validation_0-rmse:40212.8
[23]	validation_0-rmse:39089.1
[24]	validation_0-rmse:37691.1
[25]	validation_0-rmse:36875.2
[26]	validation_0-r

Cleaning up job fairing-job-rk6wl...


'fairing-job-rk6wl'

### Deploy the trained model to Kubeflow for predictions

Import the `PredictionEndpoint` and use the configured backend class. Kubeflow Fairing packages the `HousingServe` class, the trained model, and the prediction endpoint's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the prediction endpoint on Kubeflow.

In [12]:
from kubeflow.fairing import PredictionEndpoint
endpoint = PredictionEndpoint(HousingServe, input_files=['trained_ames_model.dat', "requirements.txt"],
                              docker_registry=DOCKER_REGISTRY,
                              service_type='ClusterIP',
                              backend=BackendClass(build_context_source=BuildContext))
endpoint.create()

Using default base docker image: registry.hub.docker.com/library/python:3.6.8
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/opt/conda/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_7jpo_c6p
/opt/conda/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-rwzqw-ltv99 to start...
Waiting for fairing-builder-rwzqw-ltv99 to start...
Waiting for fairing-builder-rwzqw-ltv99 to start...
Pod started running True


INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0001] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0001] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0002] Built cross stage deps: map[]
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0003] Unpacking rootfs as cm

  Stored in directory: /tmp/pip-ephem-wheel-cache-k8a2kz5d/wheels/84/bf/40/2f6ef700f48401ca40e5e3dd7d0e3c0a90e064897b7fe5fc08
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8a2kz5d/wheels/f2/84/7f/e89da3ee8ce35598d6382b6389fa2ada5d66acca2422537994
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8a2kz5d/wheels/7b/dc/25/3cf0b35c129232ee596c413f13d1d1f5a8e38c427266276dfd
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8a2kz5d/wheels/93/e9/b5/1cdc3544f99a54caca13832b5afa26fd98701fe709dc049576
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8a2kz5d/wheels/d7/7a/30/d212623a4cd34f6cce400f8122b1b7af740d3440c68023d51f
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8a2kz5d/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8a2kz5d/wheels/e4/76/4d/a95b8dd7b452b69e8ed4f68b69e1b55e12c9c9624dd962b191
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8a2kz5d/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a4

INFO[0071] Using files from context: [/kaniko/buildcontext/app]
INFO[0071] COPY /app/ /app/
INFO[0071] Taking snapshot of files...


Deploying the endpoint.
Cluster endpoint: http://fairing-service-frhks.eksworkshop.svc.cluster.local:5000/predict
Prediction endpoint: http://fairing-service-frhks.eksworkshop.svc.cluster.local:5000/predict


### Call the prediction endpoint
Create a test dataset, then call the endpoint on Kubeflow for predictions.

In [13]:
# Get sample data and query endpoint
(train_X, train_y), (test_X, test_y) = read_input("ames_dataset/train.csv")

**Wait for the service to start and replace `<endpoint>` with the output from last step.**

In [15]:
# PR https://github.com/kubeflow/fairing/pull/376
# Add `:5000/predict` to mitigate the issue.

#################
endpoint.url='http://fairing-service-frhks.eksworkshop.svc.cluster.local:5000/predict'
#################

endpoint.predict_nparray(test_X)

{'data': {'names': [],
  'tensor': {'shape': [365],
   'values': [165164.875,
    111924.984375,
    191548.140625,
    104526.6171875,
    179303.25,
    84702.0546875,
    118877.984375,
    132294.71875,
    133606.21875,
    107332.8515625,
    321764.0,
    182190.046875,
    230963.40625,
    174228.75,
    290951.1875,
    177461.453125,
    200190.75,
    122958.15625,
    133376.265625,
    123258.2734375,
    285932.1875,
    201018.390625,
    133680.75,
    130342.6328125,
    127097.6015625,
    118401.671875,
    215022.828125,
    84957.3671875,
    102413.1953125,
    176742.28125,
    118672.8125,
    209591.78125,
    236567.546875,
    217784.34375,
    122548.890625,
    160624.375,
    131057.703125,
    134156.1875,
    249800.1875,
    174228.75,
    114823.9296875,
    129791.625,
    98600.8046875,
    180586.625,
    126388.6328125,
    124807.3125,
    166880.390625,
    355064.84375,
    109526.2578125,
    94474.359375,
    126749.578125,
    170985.75,
   

### Clean up the prediction endpoint
Delete the prediction endpoint created by this notebook.

In [16]:
endpoint.delete()

Deleting the endpoint. 
Deleted service: eksworkshop/fairing-service-frhks
Deleted deployment: eksworkshop/fairing-deployer-qgxbg
